In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
from data_generation import ChessPositionGen

import datetime
%load_ext tensorboard

In [3]:
# Setting paramaters on early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          mode='min',
                          restore_best_weights=True)

log_dir = "logs/fit/full_puzzle" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [4]:
# Memory management, likely not necessary, but used as a safety as per the documentation recommendations on using GPUS

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [5]:
train = pd.read_csv('fens/train.csv')
val = pd.read_csv('fens/val.csv')

In [6]:
train_gen = ChessPositionGen(train, batch_size=512)
val_gen = ChessPositionGen(val, batch_size=512)

In [7]:
full_puzzle_model = models.Sequential()
full_puzzle_model.add(layers.Conv2D(64, 4, padding='same', input_shape=(8,8,13), activation='relu'))
full_puzzle_model.add(layers.MaxPooling2D(2))
full_puzzle_model.add(layers.Conv2D(32, 2, padding='same', activation='relu'))
full_puzzle_model.add(layers.Flatten())
full_puzzle_model.add(layers.Dense(64, activation='relu'))
full_puzzle_model.add(layers.Dense(1, activation='sigmoid'))
full_puzzle_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc'])
full_puzzle_model.summary()

# Fitting the model
full_puzzle_history = full_puzzle_model.fit(x=train_gen,
                    validation_data=val_gen,
                    # steps_per_epoch=100,
                    epochs=30,
                    callbacks=[earlystop, tensorboard_callback]
                    )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 64)          13376     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 32)          8224      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 54,497
Trainable params: 54,497
Non-trainable params: 0
____________________________________________________

In [10]:
full_puzzle_model.save('full-puzzle-PB')

INFO:tensorflow:Assets written to: full-puzzle-PB/assets


In [9]:
# test = pd.read_csv('fens/test.csv')
# test_gen = ChessPositionGen(test)

In [ ]:
%tensorboard --logdir logs/fit
